<a href="https://colab.research.google.com/github/kuheli31/Fake-News-Detector/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import re
import string
import requests
from requests.utils import quote
from urllib.parse import quote

# Enter the news to verify if its real or fake

In [17]:
news_check = input("Enter the news to verify if it is real or fake: ")

Enter the news to verify if it is real or fake: Google parent Alphabet Inc. said it will cut about 12,000 jobs, more than 6% of its global workforce, becoming the latest tech giant to retrench after years of abundant growth and hiring.


# Data Collection

**G-News API (*Real News*)**

In [ ]:
#Real time news api (https://gnews.io/dashboard) -- 100 requests per day
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
#Endpoints are -> {search , top-headlines}
endpoint = 'search'
#Parameters are -> {The topic the user want to search}
question = input('Enter your question: ')
parameters = f'q={quote(question)}&lang=en&max=5'
#Example (try this link to understand the url)-> https://gnews.io/api/v4/top-headlines?q=modi&lang=en&max=5&apikey=d2b27a1e3f818564545508b6c744d24a
news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'

response = requests.get(news_api_url)
data = response.json()

if 'articles' in data:
    for article in data['articles']:
        print(f"Published Time: {article['publishedAt']}")
        print(f"Title: {article['title']}")
        print(f"Description: {article['description']}")
        print(f"Image: {article['image']}")
        print(f"Content: {article['content']}")
        print(f"Source Name: {article['source']['name']}")
        print(f"Source URL: {article['source']['url']}")
        print(f"Source Country: {article['source']['country']}")
        print(f"URL: {article['url']}\n")
else:
    print(data)

Enter your question: trump news
Published Time: 2025-10-15T20:30:16Z
Title: Fox News Politics Newsletter: Military funding on the line
Description: Welcome to the Fox News Politics newsletter, with the latest updates on the Trump administration, Capitol Hill and more Fox News politics content.
Image: https://static.foxnews.com/foxnews.com/content/uploads/2025/10/mike-johnson-shutdown-oct-2025.jpg
Content: NEWYou can now listen to Fox News articles!
Welcome to the Fox News Politics newsletter, with the latest updates on the Trump administration, Capitol Hill and more Fox News politics content. Here's what's happening…
-Republicans fume as Democrats blo... [2781 chars]
Source Name: Fox News
Source URL: https://www.foxnews.com
Source Country: us
URL: https://www.foxnews.com/politics/fox-news-politics-newsletter-october-13th-draft

Published Time: 2025-10-15T20:25:44Z
Title: Trump says Israel could resume fighting in Gaza if Hamas doesn't uphold ceasefire
Description: “Israel will return t

**Twitter API**

In [ ]:
# Importing necessary libraries for X (Twitter) API interaction
import tweepy

# Hardcoded Bearer Token
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

#Searching in tweets
question = input('Enter your search query for X (Twitter): ').strip()

# --- Main Execution Block ---
if not X_API_BEARER_TOKEN or not question:
    print("ERROR: Both the Bearer Token and the search query are required.")
else:
    try:
        # Initialize Tweepy Client
        client = tweepy.Client(X_API_BEARER_TOKEN)

        # API Call with image support
        response = client.search_recent_tweets(
            query=f'{question} -is:retweet has:images lang:en',
            max_results=10,
            tweet_fields=['created_at', 'public_metrics', 'attachments'],
            user_fields=['name', 'username', 'verified'],
            media_fields=['url', 'preview_image_url', 'type'],
            expansions=['author_id', 'attachments.media_keys']
        )

        if not response or not response.data:
            print("\nNo recent tweets found for that query.")
        else:
            tweets = response.data
            users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
            media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

            print("\n--- Found Tweets ---\n")
            for tweet in tweets:
                author = users.get(tweet.author_id)
                media_urls = []

                # Extract media URLs
                if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
                    for key in tweet.attachments['media_keys']:
                        if key in media and media[key].type == 'photo':
                            media_urls.append(media[key].url)

                tweet_url = f"https://x.com/{author.username}/status/{tweet.id}" if author else None
                author_url = f"https://x.com/{author.username}" if author else None

                print(f"Published Time: {tweet.created_at}")
                print(f"Tweet Text: {tweet.text}")
                if author:
                    print(f"Author Name: {author.name} (@{author.username})")
                    print(f"Author URL: {author_url}")
                print(f"Tweet URL: {tweet_url}")

                if media_urls:
                    print("Image URLs:")
                    for img in media_urls:
                        print(f"- {img}")
                else:
                    print("No images found.")

                print("\n")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---")
        print("The Bearer Token you provided is invalid. Please check your credentials on the X Developer Portal.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---")
        print("Your search query is invalid or too complex. Please check the format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")


Enter your search query for X (Twitter): moon

An unexpected error occurred.
Error details: 429 Too Many Requests
Too Many Requests


**Reddit API**

In [ ]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries for Reddit API interaction
import praw
import datetime # To format the timestamp from Reddit

# --- API Configuration ---
REDDIT_CLIENT_ID = 'utnSmiRRrv9ZDIlJ9LveMg'
REDDIT_CLIENT_SECRET = 'jzP9XRolmRsu3CNiDZmKF0IDDGcXpA'
REDDIT_USER_AGENT = 'python:moonsearcher:v1.0 (by /u/Dependent_Car_5879)'

# --- Search Parameters ---
try:
    question = input('Enter your search query for Reddit: ').strip()
    limit_str = input('How many results would you like? (e.g., 10): ').strip()
    num_results = int(limit_str) if limit_str.isdigit() else 10
except EOFError:
    print("\nNo input provided for search query. Exiting.")
    exit()

# --- Main Execution Block ---
if not REDDIT_CLIENT_ID or not REDDIT_CLIENT_SECRET or not question:
    print("ERROR: Client ID, Client Secret, and a search query are required.")
else:
    try:
        reddit = praw.Reddit(
            client_id=REDDIT_CLIENT_ID,
            client_secret=REDDIT_CLIENT_SECRET,
            user_agent=REDDIT_USER_AGENT,
        )

        search_results = reddit.subreddit("all").search(
            query=question,
            sort='new',
            limit=num_results
        )

        submissions = list(search_results)

        if not submissions:
            print("\nNo recent posts found for that query.")
        else:
            print(f"\n--- Found {len(submissions)} Posts ---\n")
            for submission in submissions:
                author_name = submission.author.name if submission.author else "[deleted]"
                post_url = f"https://www.reddit.com{submission.permalink}"
                author_url = f"https://www.reddit.com/user/{author_name}" if author_name != "[deleted]" else "N/A"

                published_time = datetime.datetime.fromtimestamp(submission.created_utc)

                # --- Print basic info ---
                print(f"Published Time: {published_time.strftime('%Y-%m-%d %H:%M:%S')}")
                print(f"Post Title: {submission.title}")
                print(f"Subreddit: r/{submission.subreddit.display_name}")
                print(f"Score: {submission.score} (Upvote Ratio: {submission.upvote_ratio * 100:.0f}%)")
                print(f"Author: u/{author_name}")
                print(f"Author URL: {author_url}")
                print(f"Post URL: {post_url}")

                # --- Print full text if available ---
                if submission.selftext:
                    print(f"Post Text: {submission.selftext.replace(chr(10), ' ')}")

                # --- Extract image if any ---
                image_url = None
                if submission.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                    image_url = submission.url
                elif hasattr(submission, 'preview'):
                    try:
                        image_url = submission.preview['images'][0]['source']['url']
                    except (KeyError, IndexError):
                        pass

                if image_url:
                    print(f"Image URL: {image_url}")
                else:
                    print("Image: None")

                print("\n" + "-"*60 + "\n")

    except praw.exceptions.PRAWException as e:
        print("\n--- API ERROR ---")
        print("An error occurred with the Reddit API. This could be due to invalid credentials or a connection problem.")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred.")
        print(f"Error details: {e}")


Enter your search query for Reddit: roads india
How many results would you like? (e.g., 10): 10


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Found 10 Posts ---

Published Time: 2025-10-16 06:35:51
Post Title: Misled by Triumph showroom about price hike — lost ₹20-25k before Diwali offers
Subreddit: r/triumph4India
Score: 1 (Upvote Ratio: 100%)
Author: u/cipherthread
Author URL: https://www.reddit.com/user/cipherthread
Post URL: https://www.reddit.com/r/triumph4India/comments/1o7yzfm/misled_by_triumph_showroom_about_price_hike_lost/
Post Text:  I went to the Triumph showroom on 10th September after hearing that prices would increase after 22nd September due to a GST revision. The showroom staff also confirmed this and advised me to book the bike before 22nd to avoid the price hike.  So I paid 100k on 14th September and they completed the RTO registration and insurance by 20th September later i found out that the prices actually didn't increase after 22nd September and in fact Triumph launched Diwali offers in October.  I had clearly told them that I wanted delivery on 18th October (Dhanteras), but they still pushed me t

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-16 04:14:42
Post Title: Some people deserve to be poor
Subreddit: r/pune
Score: 10 (Upvote Ratio: 66%)
Author: u/ayxsh_03
Author URL: https://www.reddit.com/user/ayxsh_03
Post URL: https://www.reddit.com/r/pune/comments/1o7wmjw/some_people_deserve_to_be_poor/
Post Text: Sometimes I genuinely feel that some people deserve to be poor because of the way they behave. In India, a shocking number of people lack even the most basic civic sense, and the rest of us end up paying the price for their ignorance and arrogance. Just yesterday, I faced three separate incidents in traffic that made me lose faith in people's sense of responsibility. Even after giving the correct left indicator, a scooty guy behind me kept honking aggressively, glaring at me, and even threatening to beat me up. It's unbelievable how people can be so impatient and uncivilized. Later, at a green signal that clearly allowed u

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Image: None

------------------------------------------------------------

Published Time: 2025-10-16 03:56:23
Post Title: Fix your country
Subreddit: r/IndianTeenagers
Score: 1 (Upvote Ratio: 100%)
Author: u/No-Match-6725
Author URL: https://www.reddit.com/user/No-Match-6725
Post URL: https://www.reddit.com/r/IndianTeenagers/comments/1o7wafr/fix_your_country/
Post Text: I just wanna say maturing is realising how diverse and big india actually is , and leave the motherland is the retarded way of solving shit ,   I have heard of some garbage cafe where you get food in exchange of plastic waste and many other thing , and I have seen individuals and organisations working to clean or country by all means of fixing , maybe it's the minorities of people that runs   idk why I am Posting this here but it just feels right , be aware, save your trash and put it in the right, not on the side of the road . If you want high living standards make it yourself, it for everyone and throw trash and make

# **Data Preprocessing**

In [5]:
!pip install emoji
!pip install beautifulsoup4 spacy scikit-learn
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 112.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


*Real news data in csv file*

In [6]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import emoji
import re
import pandas as pd

# --- GNews API Config ---
news_api_key = 'd2b27a1e3f818564545508b6c744d24a'
endpoint = 'search'

# --- Functions ---
def clean_text(text):
    """Clean text: remove HTML, emojis, normalize whitespace, lowercase."""
    if not text:
        return ""
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Lowercase
    return text.lower().strip()

def fetch_news(query, max_results=5):
    """Fetch news articles from GNews API."""
    parameters = f'q={quote(query)}&lang=en&max={max_results}'
    news_api_url = f'https://gnews.io/api/v4/{endpoint}?{parameters}&apikey={news_api_key}'
    response = requests.get(news_api_url)
    data = response.json()
    return data.get('articles', [])

def preprocess_articles(articles):
    """Preprocess articles and return as a DataFrame."""
    processed = []
    for article in articles:
        processed.append({
            "published_at": article.get('publishedAt', ''),
            "title": clean_text(article.get('title', '')),
            "description": clean_text(article.get('description', '')),
            "content": clean_text(article.get('content', '')),
            "image": article.get('image', ''),
            "source_name": article['source'].get('name', ''),
            "source_url": article['source'].get('url', ''),
            "source_country": article['source'].get('country', ''),
            "article_url": article.get('url', '')
        })
    return pd.DataFrame(processed)

# --- Main ---
question = news_check

# Fetch articles
articles = fetch_news(question)

if not articles:
    print("No articles found or API limit reached.")
else:
    # Preprocess
    df = preprocess_articles(articles)

    # Display cleaned data
    print(df)

    # Optional: save preprocessed data
    df.to_csv('preprocessed_news.csv', index=False)
    print("\nData saved to 'preprocessed_news.csv'.")


No articles found or API limit reached.


*Twitter datas in csv file*

In [8]:
# --- Import libraries ---
import pandas as pd
import re
import emoji
from bs4 import BeautifulSoup
import tweepy
import spacy

# --- Load SpaCy model for keyword extraction ---
nlp = spacy.load("en_core_web_sm")

# --- Your X API Bearer Token ---
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

# --- Functions ---

def clean_text(text):
    """Clean text: remove HTML, emojis, normalize whitespace, lowercase."""
    if not text:
        return ""
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Lowercase
    return text.lower().strip()

def extract_keywords(sentence):
    """Extract important keywords from a full sentence."""
    doc = nlp(sentence)
    keywords = []
    for token in doc:
        if token.is_stop:
            continue
        if token.pos_ in ['NOUN', 'PROPN']:  # nouns and proper nouns
            keywords.append(token.text)
    return " ".join(keywords)

def fetch_tweets(query, max_results=20):
    """Fetch recent tweets using X API (Tweepy)."""
    client = tweepy.Client(X_API_BEARER_TOKEN)
    response = client.search_recent_tweets(
        query=f'{query} lang:en',  # only English tweets
        max_results=max_results,
        tweet_fields=['created_at', 'public_metrics', 'attachments'],
        user_fields=['name', 'username', 'verified'],
        media_fields=['url', 'preview_image_url', 'type'],
        expansions=['author_id', 'attachments.media_keys']
    )
    return response

def preprocess_tweets(response):
    """Preprocess tweets and return a DataFrame."""
    if not response or not response.data:
        return pd.DataFrame()  # empty dataframe if no tweets

    tweets = response.data
    users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
    media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

    processed = []
    for tweet in tweets:
        author = users.get(tweet.author_id)
        media_urls = []

        if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
            for key in tweet.attachments['media_keys']:
                if key in media and media[key].type == 'photo':
                    media_urls.append(media[key].url)

        processed.append({
            "published_at": tweet.created_at,
            "tweet_text": clean_text(tweet.text),
            "author_name": author.name if author else "",
            "author_username": author.username if author else "",
            "author_verified": author.verified if author else False,
            "tweet_url": f"https://x.com/{author.username}/status/{tweet.id}" if author else "",
            "author_url": f"https://x.com/{author.username}" if author else "",
            "image_urls": media_urls
        })

    return pd.DataFrame(processed)

# --- Main Execution ---

news_check = input("Enter the news to verify if it is real or fake: ").strip()

if not X_API_BEARER_TOKEN or not news_check:
    print("ERROR: Both the Bearer Token and the news sentence are required.")
else:
    try:
        # Step 1: Extract keywords
        keywords_query = extract_keywords(news_check)
        print(f"Searching tweets using keywords: {keywords_query}")

        # Step 2: Fetch tweets using keywords
        response = fetch_tweets(keywords_query)

        # Step 3: Preprocess and display tweets
        df = preprocess_tweets(response)
        if df.empty:
            print("\nNo recent tweets found for the given news keywords.")
        else:
            print(df)

            # Optional: Save preprocessed tweets for ML/NLP
            df.to_csv('preprocessed_tweets.csv', index=False)
            print("\nData saved to 'preprocessed_tweets.csv'.")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---\nCheck your Bearer Token.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---\nCheck your query format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


Enter the news to verify if it is real or fake: Donald Trump raised tariffs on Indian goods to 50%, with the new rate set to go into effect in 21 days. 
Searching tweets using keywords: Donald Trump tariffs goods % rate effect days

An unexpected error occurred: 429 Too Many Requests
Too Many Requests


In [29]:
# --- Import libraries ---
import pandas as pd
import re
import emoji
from bs4 import BeautifulSoup
import tweepy
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Load SpaCy model for keyword extraction ---
nlp = spacy.load("en_core_web_sm")

# --- Your X API Bearer Token ---
X_API_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAKCJ4QEAAAAAcEQy2tIw03BHpMFUg3fg9sbpGu8%3Db2Q2D7tbJUI2A0fJjOOMz69iKILmCmXeBYwVNNoxOaty3y2Oj2"

# --- Functions ---

def clean_text(text):
    """Clean text: remove HTML, emojis, normalize whitespace, lowercase."""
    if not text:
        return ""
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    # Lowercase
    return text.lower().strip()

def extract_keywords_tfidf(text, top_n=15):
    """Extract keywords using TF-IDF from the news text."""
    # Clean text
    text_clean = clean_text(text)
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text_clean])
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray()[0]
    # Pick top N keywords
    top_indices = scores.argsort()[-top_n:][::-1]
    top_keywords = [feature_names[i] for i in top_indices]
    return top_keywords

def clean_keywords_for_query(keywords):
    """Clean keywords to avoid special character issues for X API search."""
    cleaned = [re.sub(r'[^A-Za-z0-9]', '', kw) for kw in keywords if kw.strip()]
    return [kw for kw in cleaned if kw]  # remove empty strings

def fetch_tweets(query, max_results=20):
    """Fetch recent tweets using X API (Tweepy)."""
    client = tweepy.Client(X_API_BEARER_TOKEN)
    response = client.search_recent_tweets(
        query=f'{query} lang:en',  # only English tweets
        max_results=max_results,
        tweet_fields=['created_at', 'public_metrics', 'attachments'],
        user_fields=['name', 'username', 'verified'],
        media_fields=['url', 'preview_image_url', 'type'],
        expansions=['author_id', 'attachments.media_keys']
    )
    return response

def preprocess_tweets(response, news_keywords):
    """Preprocess tweets and return a DataFrame with relevance score."""
    if not response or not response.data:
        return pd.DataFrame()  # empty dataframe if no tweets

    tweets = response.data
    users = {user['id']: user for user in response.includes.get('users', [])} if response.includes else {}
    media = {m['media_key']: m for m in response.includes.get('media', [])} if response.includes else {}

    processed = []
    for tweet in tweets:
        author = users.get(tweet.author_id)
        media_urls = []

        if hasattr(tweet, 'attachments') and 'media_keys' in tweet.attachments:
            for key in tweet.attachments['media_keys']:
                if key in media and media[key].type == 'photo':
                    media_urls.append(media[key].url)

        tweet_text_clean = clean_text(tweet.text)

        # Compute relevance using TF-IDF cosine similarity
        combined_texts = [' '.join(news_keywords), tweet_text_clean]
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(combined_texts)
        relevance = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

        processed.append({
            "published_at": tweet.created_at,
            "tweet_text": tweet_text_clean,
            "author_name": author.name if author else "",
            "author_username": author.username if author else "",
            "author_verified": author.verified if author else False,
            "tweet_url": f"https://x.com/{author.username}/status/{tweet.id}" if author else "",
            "author_url": f"https://x.com/{author.username}" if author else "",
            "image_urls": media_urls,
            "relevance": relevance
        })

    # Sort by relevance
    df = pd.DataFrame(processed)
    df = df.sort_values(by='relevance', ascending=False).reset_index(drop=True)
    return df

# --- Main Execution ---

news_check = input("Enter the news to verify if it is real or fake: ").strip()

if not X_API_BEARER_TOKEN or not news_check:
    print("ERROR: Both the Bearer Token and the news sentence are required.")
else:
    try:
        # Step 1: Extract keywords using TF-IDF
        news_keywords = extract_keywords_tfidf(news_check, top_n=15)
        # Clean keywords for X search
        keywords_query = clean_keywords_for_query(news_keywords)
        query_string = ' '.join(keywords_query)
        print(f"Searching tweets using cleaned keywords: {query_string}")

        # Step 2: Fetch tweets using keywords
        response = fetch_tweets(query_string, max_results=50)

        # Step 3: Preprocess tweets and compute relevance
        df = preprocess_tweets(response, news_keywords)
        if df.empty:
            print("\nNo recent tweets found for the given news keywords.")
        else:
            print(df)

            # Optional: Save preprocessed tweets for ML/NLP
            df.to_csv('preprocessed_tweets.csv', index=False)
            print("\nData saved to 'preprocessed_tweets.csv'.")

    except tweepy.errors.Unauthorized:
        print("\n--- AUTHENTICATION ERROR ---\nCheck your Bearer Token.")
    except tweepy.errors.BadRequest:
        print("\n--- INVALID SEARCH QUERY ---\nCheck your query format.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")


Enter the news to verify if it is real or fake: Google parent Alphabet Inc. said it will cut about 12,000 jobs, more than 6% of its global workforce, becoming the latest tech giant to retrench after years of abundant growth and hiring.
Searching tweets using cleaned keywords: years workforce tech said retrench parent latest jobs hiring growth google global giant cut alphabet

An unexpected error occurred: 429 Too Many Requests
Too Many Requests


*Reddit's datas in csv file*

In [13]:
!pip install praw
!pip install rake_nltk


In [27]:
import praw
import datetime
from bs4 import BeautifulSoup
import emoji
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Reddit API Configuration ---
REDDIT_CLIENT_ID = 'utnSmiRRrv9ZDIlJ9LveMg'
REDDIT_CLIENT_SECRET = 'jzP9XRolmRsu3CNiDZmKF0IDDGcXpA'
REDDIT_USER_AGENT = 'python:moonsearcher:v1.0 (by /u/Dependent_Car_5879)'

# --- SpaCy NER ---
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
    USE_SPACY = True
except:
    USE_SPACY = False
    print("⚠ spaCy model not found. Falling back to regex-based extraction.")

# ----------------- Utilities -----------------

def clean_text(text):
    if not text:
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def extract_keywords(text):
    if not text:
        return []
    keywords = set()
    # Numbers / Percentages
    keywords.update(re.findall(r'\b\d+(\.\d+)?%\b', text))
    keywords.update(re.findall(r'\b\d{1,6}\b', text))
    # Dates
    keywords.update(re.findall(r'\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\.?\s+\d{1,2},?\s*\d{0,4}\b', text, re.IGNORECASE))

    if USE_SPACY:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ in ["PERSON","ORG","GPE","DATE","TIME","PERCENT","MONEY","QUANTITY"]:
                keywords.add(ent.text)
    else:
        # Fallback names
        name_matches = re.findall(r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)\b', text)
        keywords.update(name_matches)

    return list(set([kw.strip() for kw in keywords if len(kw) > 1]))

# ----------------- Reddit Fetch -----------------

def fetch_reddit_posts(query, max_posts=500):
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent=REDDIT_USER_AGENT
    )

    subreddit = reddit.subreddit("all")
    submissions = []

    for submission in subreddit.search(query=query, sort='new', limit=None):
        submissions.append(submission)
        if len(submissions) >= max_posts:
            break

    print(f"Fetched {len(submissions)} posts from Reddit.")
    return submissions

# ----------------- Relevance -----------------

def compute_relevance(news_keywords, post_text):
    combined_texts = [' '.join(news_keywords), post_text]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(combined_texts)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    return similarity[0][0]

# ----------------- Preprocess -----------------

def preprocess_reddit_posts(submissions, news_keywords):
    processed = []
    for submission in submissions:
        author_name = submission.author.name if submission.author else "[deleted]"
        post_url = f"https://www.reddit.com{submission.permalink}"
        author_url = f"https://www.reddit.com/user/{author_name}" if author_name != "[deleted]" else "N/A"
        published_time = datetime.datetime.fromtimestamp(submission.created_utc)
        full_text = clean_text(submission.title + " " + submission.selftext)
        post_keywords = extract_keywords(full_text)
        relevance = compute_relevance(news_keywords, full_text)
        relevant_keywords = [kw for kw in post_keywords if kw.lower() in [k.lower() for k in news_keywords]]
        processed.append({
            "published_at": published_time.strftime('%Y-%m-%d %H:%M:%S'),
            "title": clean_text(submission.title),
            "selftext": clean_text(submission.selftext),
            "subreddit": submission.subreddit.display_name,
            "score": submission.score,
            "upvote_ratio": submission.upvote_ratio,
            "author_name": author_name,
            "author_url": author_url,
            "post_url": post_url,
            "keywords": relevant_keywords,
            "relevance": relevance
        })

    # Sort by relevance descending
    processed = sorted(processed, key=lambda x: x['relevance'], reverse=True)
    return pd.DataFrame(processed)

# ----------------- Main -----------------

news_text = news_check
news_keywords = extract_keywords(news_text)
query = ' '.join(news_keywords)

submissions = fetch_reddit_posts(query, max_posts=500)
df = preprocess_reddit_posts(submissions, news_keywords=news_keywords)

df.to_csv('relevant_reddit_posts.csv', index=False)
print(df)


Fetched 246 posts from Reddit.
            published_at                                              title  \
0    2025-10-16 07:25:26             wealth tax and housing reform proposal   
1    2025-10-16 11:47:33                               projectionlab vs swr   
2    2025-10-16 11:22:11            12 days ago my head started hurting....   
3    2025-10-16 05:21:44           i've been shadowbanned for over 6 months   
4    2025-10-16 09:04:43  £75m package to train 12,000 construction work...   
..                   ...                                                ...   
241  2025-10-16 03:26:49                              from the hindu today.   
242  2025-10-16 02:14:24  "the talented mr. epstein" - 2003 article on j...   
243  2025-10-16 01:47:50                                    today’s pickups   
244  2025-10-16 01:40:26               just found these at local book store   
245  2025-10-16 00:20:59  every note in the new poppy playtime map on fo...   

                    

In [26]:
# Filter posts with relevance above a threshold, e.g., 0.05
relevant_posts = df[df['relevance'] > 0.05].sort_values(by='relevance', ascending=False)
relevant_posts.to_csv('filtered_reddit_posts.csv', index=False)
print(relevant_posts)


           published_at                                              title  \
0   2025-10-16 07:25:26             wealth tax and housing reform proposal   
1   2025-10-16 11:47:33                               projectionlab vs swr   
2   2025-10-16 11:22:11            12 days ago my head started hurting....   
3   2025-10-16 05:21:44           i've been shadowbanned for over 6 months   
4   2025-10-16 09:04:43  £75m package to train 12,000 construction work...   
5   2025-10-15 22:41:41     what will happen if interest rates fall to 6%?   
6   2025-10-16 09:31:37  my body is deteriorating - i can't handle it a...   
7   2025-10-16 00:14:07  engineers now make less than firefighters in t...   
8   2025-10-16 08:31:22  suggestions, tips and tricks for a new pixel user   
9   2025-10-16 07:40:32  how i self-hosted my saas with just $12, for a...   
10  2025-10-16 10:25:59  england - hr - training agreement and notice q...   
11  2025-10-16 04:05:45  [wts] amouage outlands, ysl 6 place sai

In [ ]:
#Creating Dataframe
real_news = pd.read_csv('preprocessed_news.csv')
twitter_posts = pd.read_csv('preprocessed_tweets.csv')
reddit_posts = pd.read_csv('preprocessed_reddit_posts.csv')

In [ ]:
real_news

,published_at,title,description,content,image,source_name,source_url,source_country,article_url
0,2025-10-15T18:05:41Z,nagaland’s ruling alliance pda hails modi govt...,nagaland’s ruling alliance pda hails modi govt...,"kohima, oct 15 the ruling people’s democratic ...",https://d3pc1xvrcw35tl.cloudfront.net/ln/feedI...,Lokmat Times,https://www.lokmattimes.com,in,https://www.lokmattimes.com/national/nagalands...
1,2025-10-15T17:56:00Z,gst reforms increased purchasing power of cons...,arunachal pradesh deputy cm chowna mein highli...,arunachal pradesh deputy cm chowna mein highli...,https://static.toiimg.com/thumb/msid-124586903...,Times of India,https://timesofindia.indiatimes.com,in,https://timesofindia.indiatimes.com/city/guwah...
2,2025-10-15T17:44:15Z,will pm modi convince his 'good friend' in us ...,congress leader and ex-environment minister qu...,congress leader jairam ramesh on wednesday cha...,https://media.assettype.com/outlookindia/2024-...,Outlook India,https://www.outlookindia.com,in,https://www.outlookindia.com/national/will-pm-...
3,2025-10-15T17:16:41Z,big worry for donald trump as pm modi and pres...,"according to the reports, the goal of the meet...","according to the reports, the goal of the meet...",https://static.india.com/wp-content/uploads/20...,India.com,https://www.india.com,in,https://www.india.com/news/world/donald-trump-...
4,2025-10-15T17:11:44Z,"'ekjut nda, ekjut bihar': ahead of polls, pm m...",prime minister narendra modi urges bihar women...,"last updated:october 15, 2025, 22:41 ist prime...",https://images.news18.com/ibnlive/uploads/2025...,News18,https://www.news18.com,in,https://www.news18.com/elections/ekjut-nda-ekj...


In [ ]:
twitter_posts

,published_at,tweet_text,author_name,author_username,author_verified,tweet_url,author_url,image_urls
0,2025-10-16 07:38:26+00:00,breaking: trump says modi told him that india ...,Ounka,OunkaOnX,False,https://x.com/OunkaOnX/status/1978727218252513742,https://x.com/OunkaOnX,['https://pbs.twimg.com/media/G3XZ6SZXkAA3UQa....
1,2025-10-16 07:38:21+00:00,pm modi's dedication to serving the nation is ...,Rekanjadhav,rekana24,False,https://x.com/rekana24/status/1978727197130252694,https://x.com/rekana24,['https://pbs.twimg.com/media/G3XZ4NSWcAAq0cQ....
2,2025-10-16 07:38:11+00:00,modi’s path to viksit bharat goes through its ...,Dhruvkishor Patil,DhruvkishorP,False,https://x.com/DhruvkishorP/status/197872715528...,https://x.com/DhruvkishorP,['https://pbs.twimg.com/media/G3XZ25PWkAEXfPi....
3,2025-10-16 07:38:01+00:00,"the story of two, bold and clear, mallanna’s v...",aakas,aakas27366,False,https://x.com/aakas27366/status/19787271147542...,https://x.com/aakas27366,['https://pbs.twimg.com/media/G3XMZhDXkAARSHl....
4,2025-10-16 07:37:48+00:00,"mallanna stands with fearless tone, facing pow...",vikas,vikas26366,False,https://x.com/vikas26366/status/19787270594603...,https://x.com/vikas26366,['https://pbs.twimg.com/media/G3XMZhDXkAARSHl....
5,2025-10-16 07:37:44+00:00,the modi government’s new wave of war talk aga...,MISHAL KHAN,MishalKhanNDU12,False,https://x.com/MishalKhanNDU12/status/197872704...,https://x.com/MishalKhanNDU12,['https://pbs.twimg.com/media/G3XZVDiWMAA6TtD....
6,2025-10-16 07:37:41+00:00,in response to u.s. president donald trump's c...,Peek TV,PeekTV_in,False,https://x.com/PeekTV_in/status/197872703068740...,https://x.com/PeekTV_in,['https://pbs.twimg.com/media/G3XZvruXAAAjgDk....
7,2025-10-16 07:37:35+00:00,@rapidresponse47 @potus modi to trump: https:/...,vadapav ugh,magnoninsig,False,https://x.com/magnoninsig/status/1978727006406...,https://x.com/magnoninsig,['https://pbs.twimg.com/media/G3XZuRfW4AA2Pax....
8,2025-10-16 07:37:34+00:00,"mallanna stands with fearless tone, facing pow...",varun,varunjaat7171,False,https://x.com/varunjaat7171/status/19787270020...,https://x.com/varunjaat7171,['https://pbs.twimg.com/media/G3XMZhDXkAARSHl....
9,2025-10-16 07:37:23+00:00,guys do you know that our pm narendra modi is ...,Rintu Pawar,rintu_off,False,https://x.com/rintu_off/status/197872695404072...,https://x.com/rintu_off,['https://pbs.twimg.com/media/G3XZrbEXcAAd7RC....


In [ ]:
reddit_posts

,published_at,title,selftext,subreddit,score,upvote_ratio,author_name,author_url,post_url,image_url
0,2025-10-16 07:24:56,do you agree?,NaN,IndiaMemes,3,1.00,Calm_Dash250,https://www.reddit.com/user/Calm_Dash250,https://www.reddit.com/r/IndiaMemes/comments/1...,https://i.redd.it/1gnabsw1efvf1.jpeg
1,2025-10-16 07:17:53,us president donald trump claimed that prime m...,"speaking to reporters on october 15, trump sai...",BusinessTodayNews,1,1.00,BusinessToday,https://www.reddit.com/user/BusinessToday,https://www.reddit.com/r/BusinessTodayNews/com...,NaN
2,2025-10-16 07:16:09,qualcuno ha già parlato di equilibrio vita-lav...,NaN,LinkedInCringeIT,5,1.00,Mondonauta0206,https://www.reddit.com/user/Mondonauta0206,https://www.reddit.com/r/LinkedInCringeIT/comm...,https://i.redd.it/gyfmp98gcfvf1.jpeg
3,2025-10-16 07:02:26,new narendra modi upload: live: pm modi perfor...,this post contains content not supported on ol...,indian_discussions,1,1.00,yt-app,https://www.reddit.com/user/yt-app,https://www.reddit.com/r/indian_discussions/co...,https://external-preview.redd.it/nw6g1ihrhcMDV...
4,2025-10-16 06:46:53,trump says modi assured him india will halt ru...,NaN,u_Virtual_Swan_6551,1,1.00,Virtual_Swan_6551,https://www.reddit.com/user/Virtual_Swan_6551,https://www.reddit.com/r/u_Virtual_Swan_6551/c...,NaN
5,2025-10-16 06:27:43,[business] - trump says modi has agreed to sto...,NaN,AutoNewspaper,1,1.00,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,https://external-preview.redd.it/6hASWTjC-71TX...
6,2025-10-16 06:27:42,[top stories] - trump says modi has agreed to ...,NaN,AutoNewspaper,1,1.00,AutoNewspaperAdmin,https://www.reddit.com/user/AutoNewspaperAdmin,https://www.reddit.com/r/AutoNewspaper/comment...,https://external-preview.redd.it/6hASWTjC-71TX...
7,2025-10-16 06:24:00,wait..... just a blank paper,NaN,IndiaMemes,3,1.00,Grumbling_Foot72,https://www.reddit.com/user/Grumbling_Foot72,https://www.reddit.com/r/IndiaMemes/comments/1...,https://i.redd.it/9a0kd5993fvf1.jpeg
8,2025-10-16 06:22:14,pawan kalyan grand welcome to pm narendra modi...,NaN,pawankalyan,4,1.00,Perfect_Sector1888,https://www.reddit.com/user/Perfect_Sector1888,https://www.reddit.com/r/pawankalyan/comments/...,https://i.redd.it/zsiru3713fvf1.jpeg
9,2025-10-16 06:17:34,perfect use of the song,NaN,delhi,2,0.55,kholeChature,https://www.reddit.com/user/kholeChature,https://www.reddit.com/r/delhi/comments/1o7yp8...,https://i.redd.it/mylyqv072fvf1.png
